In [18]:
import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time
import pandas as pd

In [6]:
CLOUD_API_KEY = "R4ZmoGqWaQyxy5370P_eEP7N4iWZpUJWpfbPzQ6IAGG0"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [77]:
def generate_access_token():
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth("bx", "bx")
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": CLOUD_API_KEY
    }
    response = requests.post(IAM_URL, data=data, headers=headers, auth=auth)
    json_data = response.json()
    iam_access_token = json_data['access_token']
    return iam_access_token

In [78]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

In [79]:
data_mart_id = '2ee58274-88bd-48b8-84b1-c8f42d40cfe0'
service_binding_id = '3cfa0171-5dc2-4928-a348-c65c05a32565'
subscription_id = '7fe7073b-e1f7-45e2-991b-04f6546b1f0b'
deployment_id = '047089bb-9fad-444e-b603-dec8a6684c4b'

URL = 'https://api.aiopenscale.cloud.ibm.com/v1/data_marts/' + data_mart_id + '/service_bindings/'+ service_binding_id + '/subscriptions/' + subscription_id + '/deployments/' + deployment_id +'/online'
URL

'https://api.aiopenscale.cloud.ibm.com/v1/data_marts/2ee58274-88bd-48b8-84b1-c8f42d40cfe0/service_bindings/3cfa0171-5dc2-4928-a348-c65c05a32565/subscriptions/7fe7073b-e1f7-45e2-991b-04f6546b1f0b/deployments/047089bb-9fad-444e-b603-dec8a6684c4b/online'

In [103]:
# feature on which we want to check the fairness
feature = 'Sex'

# the favorable prediction value
favorable = 'No Risk'

# the unfavorable prediction value
unfavorable = 'Risk'

# the unprivileged/minority group
minority = 'female'

# the privileged/majority group
majority = 'male'

In [104]:
def get_data_distribution(df, label):
    # the favorable data frame
    favorable_df = df.loc[df[label] == favorable]

    # the unfavorable data frame
    unfavorable_df = df.loc[df[label] == unfavorable]

    # the filtered minority data frame
    minority_df = df.loc[df[feature] == minority]

    # the filtered majority data frame
    majority_df = df.loc[df[feature] == majority]

    # the filtered data frame comprising of majority population getting favorable values
    minority_favorable_df = favorable_df.loc[df[feature] == minority]

    # the filtered data frame comprising of majority population getting favorable values
    majority_favorable_df = favorable_df.loc[df[feature] == majority]

    # the filtered data frame comprising of minority population getting unfavorable values
    minority_unfavorable_df = unfavorable_df.loc[df[feature] == minority]

    # the filtered data frame comprising of majority population getting unfavorable values
    majority_unfavorable_df = unfavorable_df.loc[df[feature] == majority]

    # find the rate at which minority population received favorable outcome
    favorable_minority_rate = len(minority_favorable_df) / len(minority_df)

    # find the rate at which majority population received favorable outcome
    favorable_majority_rate = len(majority_favorable_df) / len(majority_df)
    
    return minority_df, majority_df, minority_favorable_df, majority_favorable_df, minority_unfavorable_df, majority_unfavorable_df, favorable_minority_rate, favorable_majority_rate

In [87]:
# fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
# values = [
#   ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
#   ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
#   ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
#   ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
#   ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
#   ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
#   ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
#   ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
# ]

#!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/ravichamarthy/german-credit/master/credit_risk_training_modified_1711.csv
#!wget https://raw.githubusercontent.com/ravichamarthy/german-credit/master/score_payloads_1.json
#!wget https://raw.githubusercontent.com/ravichamarthy/german-credit/master/german_credit_data_biased_test.csv

--2019-12-11 14:14:49--  https://raw.githubusercontent.com/ravichamarthy/german-credit/master/credit_risk_training_modified_1711.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 947513 (925K) [text/plain]
Saving to: ‘credit_risk_training_modified_1711.csv.1’

100%[======================================>] 947,513     --.-K/s   in 0.04s   

2019-12-11 14:14:49 (22.3 MB/s) - ‘credit_risk_training_modified_1711.csv.1’ saved [947513/947513]



In [93]:
scoring_request_all = pd.read_csv("credit_risk_training_modified_1711.csv", sep=",", header=0)
scoring_request_df = scoring_request_all.sample(frac=0.5)
print('scoring_request_df length:'+str(len(scoring_request_df)))

# import io
# with io.open("score_payloads_1.json", "r", encoding = "utf-8") as my_file:
#     data_str = my_file.read()
#     payload_scoring = json.loads(data_str)

scoring_request_df length:3442


In [106]:
# the label column
label = 'Risk'

manual_minority_df, manual_majority_df, manual_minority_favorable_df, manual_majority_favorable_df, \
manual_minority_unfavorable_df, manual_majority_unfavorable_df, manual_favorable_minority_rate, manual_favorable_majority_rate = get_data_distribution(scoring_request_df, label)

manual_di_value = manual_favorable_minority_rate / manual_favorable_majority_rate

print('Number of minorities/unprivileged population in manual data(' + str(minority) + ') :' + str(len(manual_minority_df)))
print('Number of majorities/privilaged population in manual data(' + str(majority) + ') :' + str(len(manual_majority_df)))
print('Number of minorities with favorable outcomes in manual data(' + str(minority) + ') :' + str(len(manual_minority_favorable_df)))
print('Number of majorities with favorable outcomes in manual data(' + str(majority) + ') :' + str(len(manual_majority_favorable_df)))
print('Number of minorities with unfavorable outcomes in manual data(' + str(minority) + ') :' + str(len(manual_minority_unfavorable_df)))
print('Number of majorities with unfavorable outcomes in manual data(' + str(majority) + ') :' + str(len(manual_majority_unfavorable_df)))


print('The disparate impact metric value on the manual data:'+str(manual_di_value))

Number of minorities/unprivileged population in manual data(female) :840
Number of majorities/privilaged population in manual data(male) :2602
Number of minorities with favorable outcomes in manual data(female) :698
Number of majorities with favorable outcomes in manual data(male) :2403
Number of minorities with unfavorable outcomes in manual data(female) :142
Number of majorities with unfavorable outcomes in manual data(male) :199
The disparate impact metric value on the manual data:0.8997661653092365


In [108]:
fields = manual_minority_unfavorable_df.columns.tolist()
values = manual_minority_unfavorable_df.values.tolist()
payload_scoring = {"fields": fields, "values": values}

In [109]:
response = requests.post(URL, data=json.dumps(payload_scoring), headers=headers)

In [110]:
scoring_response = response.json()

fields = scoring_response['fields']
values = scoring_response['values']

In [113]:
# create a pandas data frame using the scored response
scoring_response_df = pd.DataFrame(values, columns = fields)
print(len(scoring_response_df))

# label = 'predictedLabel'

# scored_minority_df, scored_majority_df, scored_minority_favorable_df, scored_majority_favorable_df, \
# scored_minority_unfavorable_df, scored_majority_unfavorable_df, scored_favorable_minority_rate, scored_favorable_majority_rate = get_data_distribution(scoring_response_df, label)

# scored_di_value = scored_favorable_minority_rate / scored_favorable_majority_rate

# print('Number of minorities/unprivileged population in scored data(' + str(minority) + ') :' + str(len(scored_minority_df)))
# print('Number of majorities/privilaged population in scored data(' + str(majority) + ') :' + str(len(scored_majority_df)))
# print('Number of minorities with favorable outcomes in scored data(' + str(minority) + ') :' + str(len(scored_minority_favorable_df)))
# print('Number of majorities with favorable outcomes in scored data(' + str(majority) + ') :' + str(len(scored_majority_favorable_df)))
# print('Number of minorities with unfavorable outcomes in scored data(' + str(minority) + ') :' + str(len(scored_minority_unfavorable_df)))
# print('Number of majorities with unfavorable outcomes in scored data(' + str(majority) + ') :' + str(len(scored_majority_unfavorable_df)))
# print('The disparate impact metric value on the scored data:'+str(scored_di_value))


scoring_response_df[['predictedLabel', 'probability', 'debiased_prediction', 'debiased_probability']]
scoring_response_df['predictedLabel'].equals(scoring_response_df['debiased_prediction'])
df1 = scoring_response_df.query('predictedLabel != debiased_prediction')
df1

142


,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,Risk,Housing_IX,probability,Telephone_IX,features,InstallmentPlans_IX,Job_IX,EmploymentDuration_IX,ForeignWorker_IX,scoring_id
